# Run in QuantConnect

In [ ]:
from AlgorithmImports import *
class MeanReversionAlgorithm(QCAlgorithm):
    
    def Initialize(self):
        # Configuración del algoritmo
        self.SetStartDate(2020, 1, 1)
        self.SetEndDate(2023, 1, 1)
        self.SetCash(100000)
        
        # Definir variables de estrategia
        self.window = 21
        self.n_longs = 10
        self.n_shorts = 10
        self.vol = 500

        # Inicializar listas para posibles entradas y salidas
        self.long_symbols = []
        self.short_symbols = []

        # Configurar el universo con la selección de valores
        self.UniverseSettings.Resolution = Resolution.Daily
        self.AddUniverse(self.CoarseSelectionFunction, self.FineFilterSelectionFunction)
        self.SetWarmUp(self.window)

        # Establecer cuando hacer un rebalance y los cálculos
        self.Schedule.On(self.DateRules.WeekStart(), self.TimeRules.At(8, 31), self.Rebalance)
    
    # Función para filtrar por datos fundamentales
    def CoarseSelectionFunction(self, coarse):
        # Ordenar los activos por DollarVolume en orden descendente
        sorted_by_volume = sorted(coarse, key=lambda asset: asset.DollarVolume, reverse=True)
    
        # Seleccionar las primeras `self.vol` acciones con mayor volumen
        top_500_volume = sorted_by_volume[:self.vol]
    
        return [asset.Symbol for asset in top_500_volume]

    # Función para filtrar por métrica de mean reversion
    def FineFilterSelectionFunction(self, fine): 
        if self.IsWarmingUp:
            return []
        
        selected = []
        
        for asset in fine:
            # Obtener precios históricos (por ejemplo, 21 días)
            history = self.History(asset.Symbol, self.window, Resolution.Daily)
            
            # Asegurarse de que haya datos suficientes
            if not history.empty:
                # Calcular la media y desviación estándar de los últimos 21 días
                mean_price = history['close'].mean()
                std_dev = history['close'].std()
                current_price = history['close'].iloc[-1]
                
                # Calcular el mean reversion score como el z-score
                if std_dev != 0:  # Asegurarse de que la desviación estándar no sea cero
                    mean_reversion_score = abs(current_price - mean_price) / std_dev
                    selected.append((asset.Symbol, mean_reversion_score))
        
        # Ordenar por mean reversion score (de menor a mayor)
        selected = sorted(selected, key=lambda x: x[1])

        # Seleccionar los 50 activos con los z-scores más bajos para posibles longs
        self.long_symbols = [symbol for symbol, score in selected[:self.n_longs]]

        # Seleccionar los 50 activos con los z-scores más altos para posibles shorts
        self.short_symbols = [symbol for symbol, score in selected[-self.n_shorts:]]

        # Devolver las listas de longs y shorts separadas
        return self.long_symbols + self.short_symbols

    def Rebalance(self):
        """Rebalancea el portafolio basado en los datos de los factores."""
        longs = self.long_symbols
        shorts = self.short_symbols

        # Liquidar activos no seleccionados
        divest = [x.Key for x in self.Portfolio if x.Value.Invested and x.Key not in longs + shorts]
        self.ExecTrades(divest, 0)
            
        # Ejecutar operaciones de long y short
        if longs:
            long_weight = 0.5 / len(longs)
            self.ExecTrades(longs, long_weight)
            
        if shorts:
            short_weight = -0.5 / len(shorts)
            self.ExecTrades(shorts, short_weight)
            
        # Debug para mostrar el estado del portafolio
        self.Debug(f"{self.Time.date()} | Longs {len(longs)} | Shorts {len(shorts)} | Portfolio Value: {self.Portfolio.TotalPortfolioValue}")

    def ExecTrades(self, assets, target_percent):
        """Ejecuta trades para los activos especificados con el porcentaje de target."""
        for asset in assets:
            if self.Securities[asset].IsTradable and not self.Transactions.GetOpenOrders(asset):
                self.SetHoldings(asset, target_percent)
